In [67]:
import pandas as pd
import json
import googlemaps
import time
import folium

In [68]:
with open("../config/googleAPI.json") as f:
    config = json.load(f)
    GOOGLE_API_KEY = config["GOOGLE_API_KEY"]

In [69]:
import googlemaps
from datetime import datetime

# set up API key and client
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

# set up search query for "惠康" in Hong Kong
search_query = "惠康 Hong Kong"


In [70]:
def search_all():

    df = pd.DataFrame()
    results = gmaps.places(
        query=search_query,
        location=(22.3964, 114.1095),
        radius=15000,
        )
    df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
    print("Total number of results: ", len(df))

    while "next_page_token" in results:
        time.sleep(2)
        results = gmaps.places(
            query=search_query,
            location=(22.3964, 114.1095),
            radius=15000,
            page_token=results["next_page_token"],
            )
        df = pd.concat([df, pd.DataFrame(results["results"])], axis=0)
        print("Total number of results: ", len(df))
    return df

In [71]:
df = search_all()

Total number of results:  20
Total number of results:  40
Total number of results:  60


In [72]:
df_filtered = df[[ 'name', 'formatted_address','place_id', 'rating', 'user_ratings_total','types']]
df_filtered.loc[:,'latitude'] = df['geometry'].apply(lambda x: x['location']['lat'])
df_filtered.loc[:,'longitude'] = df['geometry'].apply(lambda x: x['location']['lng'])
df_filtered.reset_index(drop=True, inplace=True)
df_filtered

/var/folders/gz/rp9x0dzd5s13dyt0nv94f50h0000gn/T/ipykernel_5468/3109238641.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:,'latitude'] = df['geometry'].apply(lambda x: x['location']['lat'])
/var/folders/gz/rp9x0dzd5s13dyt0nv94f50h0000gn/T/ipykernel_5468/3109238641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:,'longitude'] = df['geometry'].apply(lambda x: x['location']['lng'])


,name,formatted_address,place_id,rating,user_ratings_total,types,latitude,longitude
0,wellcome,"125, Metro Harbour View Plaza, 8 Fuk Lee St, T...",ChIJfzdi4rAABDQRYsiJViBAKIs,3.7,259,"[supermarket, grocery_or_supermarket, food, po...",22.324410,114.160454
1,Wellcome,"84-90號 Queen's Road Central, Central, Hong Kong",ChIJ5b0oWHsABDQRJUwc57avQ94,3.8,362,"[supermarket, grocery_or_supermarket, food, po...",22.283245,114.155105
2,Wellcome,"319 Nathan Rd, Jordan, Hong Kong",ChIJG3W4guwABDQRsYiKQEyeUvU,3.6,275,"[supermarket, grocery_or_supermarket, food, po...",22.306436,114.171362
3,Wellcome,"Shop No.23, G/F, Hoi Fu Shopping Centre, 2 Hoi...",ChIJvdn9MLkABDQRbEofjcY9gsc,3.6,215,"[supermarket, grocery_or_supermarket, food, po...",22.315173,114.164415
4,Wellcome,"G/F, TG Place, 8 Shing Yip St, Kwun Tong, Hong...",ChIJabudQ0UBBDQRF2ggpcEzLiE,3.5,182,"[supermarket, grocery_or_supermarket, food, po...",22.310417,114.225745
5,Wellcome,"Far East Consortium Building, 204-206 Nathan R...",ChIJK8CLjesABDQRb5SeNhHVk1k,3.9,118,"[supermarket, grocery_or_supermarket, food, po...",22.303711,114.171948
6,Wellcome,"Shop No. 6B & No. 6C1, G/F, Ka Wo Bldg, Nos, 1...",ChIJt_IRRAEABDQRdjnnirMxhBc,3.5,20,"[supermarket, grocery_or_supermarket, food, po...",22.250047,114.148533
7,Wellcome - Causeway Bay,"25-29號 Great George St, Causeway Bay, Hong Kong",ChIJk3dQL1QABDQRM77UUNtOEYQ,4.0,969,"[supermarket, grocery_or_supermarket, food, po...",22.280759,114.185949
8,Wellcome - Lung Hang,"Hong Kong, HK HK HK 香港Shop 215, Lung Hang Shop...",ChIJBX4s-QUHBDQR-koBBpJUx10,3.7,253,"[supermarket, grocery_or_supermarket, food, po...",22.367896,114.179374
9,Wellcome,"Jade Mansion, 40 Waterloo Rd, Yau Ma Tei, Hong...",ChIJgY04H8EABDQR3h5KFWbhB9I,3.7,190,"[supermarket, grocery_or_supermarket, food, po...",22.312785,114.171019


In [73]:
df_filtered.to_csv("../data/processed/wellcomeMap.csv", index=False)

# Map Showing

In [74]:

# create a map centered at Hong Kong
m = folium.Map(location=[22.3193, 114.1694], zoom_start=11)

# iterate through the rows of the dataframe and add markers to the map
for index, row in df_filtered.iterrows():
    name = row['formatted_address']
    lat = row['latitude']
    lon = row['longitude']
    popup = folium.Popup(name)
    marker = folium.Marker(location=[lat, lon], popup=popup)
    marker.add_to(m)

# display the map
m


In [75]:
m.save("../data/processed/map.html")